# Data exploration of data in timestream db


## Initialization


### Imports


In [13]:
import datetime
from itertools import repeat
from os import makedirs
from typing import Optional, Sequence

import awswrangler as wr
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import scipy.signal as signal
from dotenv import load_dotenv
from numpy.polynomial import Polynomial

load_dotenv()


True

### Data retrieval


In [14]:
df: pd.DataFrame = wr.timestream.query('SELECT * FROM "when-to-water"."sensor-data"')
print(f"Retrieved {len(df)} records")
df_origin = df.copy()

Retrieved 12618 records


### Constants


In [15]:
SENSOR_PLANT_MAPPING: dict = {
    "PWS_1": "Goldfruchtpalme",
    "PWS_2": "Pilea",
    "PWS_3": "Drachenbaum",
}
PLANTS: tuple[str, ...] = tuple(SENSOR_PLANT_MAPPING.values())
COLORS: tuple[str, ...] = (
    "#E69F00",
    "#56B4E9",
    "#009E73",
    "#F0E442",
    "#0072B2",
    "#D55E00",
    "#CC79A7",
)
MINIMUM_MOISTURE: dict = {
    "Goldfruchtpalme": 10,
    "Pilea": 20,
    "Drachenbaum": 30,
}


## Transformations


### General


In [16]:
# set types and rename
df["time"] = pd.to_datetime(df["time"])
df.rename(
    columns={"measure_value::double": "value", "sensor_name": "plant"}, inplace=True
)
df["value"] = df["value"].astype(float)

df["plant"] = df["plant"].map(SENSOR_PLANT_MAPPING)

# drop n/a rows
df.dropna(inplace=True)

# drop power
df = df[df["measure_name"] != "power"]

# add unit to measurement name
df["measure_name"] = df["measure_name"].str.replace("_", " ") + " in " + df["unit"]
measurements = df["measure_name"].unique().tolist()

# drop unit
df.drop(columns=["unit"], inplace=True)

# remove 0 moisture
df = df[~((df["measure_name"] == "soil moisture in %") & (df["value"] == 0))]

# Resample df to hourly measures
df.set_index("time", inplace=True)
df = df.groupby(["plant", "measure_name"]).resample("H").mean().reset_index()

# transpose measure_name 
df.set_index(["time", "plant", "measure_name"], inplace=True)
df = df.unstack().reset_index()
df.columns = [" ".join(col).strip().replace("value ", "") for col in df.columns.values]
df.reset_index(drop=True, inplace=True)

# interpolate missing values
df.set_index("time", inplace=True)
for plant in PLANTS:
    df[df["plant"] == plant] = df[df["plant"] == plant].interpolate(method="time")

df.reset_index(inplace=True)


#### Trasformed data


In [29]:
transformed_figure = go.Figure(
    layout=go.Layout(
        title="Transformed values",
        width=1280,
        height=720,
        template="plotly_white",
        font=dict(family="Times New Roman", color="#000000"),
    )
)

for index, plant in enumerate(PLANTS):
    transformed_figure.add_trace(
        go.Scatter(
            x=df[df["plant"] == plant]["time"],
            y=df[df["plant"] == plant]["soil moisture in %"],
            mode="lines",
            name=plant,
            opacity=0.7,
            marker=dict(color=COLORS[index], size=3),
        )
    )

transformed_figure.update_scenes(
    dict(
        xaxis_title="Time",
        yaxis_title="soil moisture in %",
    )
)

transformed_figure.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1,
    xanchor="center",
    x=0.5,
    itemsizing='constant'
))

makedirs("./dist", exist_ok=True)
transformed_figure.write_html("./dist/peak_valley.html")

transformed_figure.show()

### Identify valleys/peaks in moisture


In [30]:
DISTANCE = 3
PROMINENCE = 2

all_peaks: list = []
all_valleys: list = []

for plant in PLANTS:
    df_plant = df[df["plant"] == plant]
    peaks = signal.find_peaks(
        df_plant["soil moisture in %"],
        distance=DISTANCE,
        prominence=PROMINENCE,
    )[0]
    valleys = signal.find_peaks(
        -df_plant["soil moisture in %"],
        distance=DISTANCE,
        prominence=PROMINENCE,
    )[0]
    # translate row to index
    all_peaks += [df_plant.index[peak] for peak in peaks]
    all_valleys += [df_plant.index[valley] for valley in valleys]

df["peak"] = df.index.isin(all_peaks)
df["valley"] = df.index.isin(all_valleys)


#### Plot for valleys/peaks


In [31]:
peak_valley_figure = go.Figure(
    layout=go.Layout(
        title="Peaks/Valleys",
        width=1280,
        height=720,
        template="plotly_white",
        font=dict(family="Times New Roman", color="#000000"),
    )
)

for index, plant in enumerate(PLANTS):
    peak_valley_figure.add_trace(
        go.Scatter(
            x=df[df["plant"] == plant]["time"],
            y=df[df["plant"] == plant]["soil moisture in %"],
            mode="lines",
            legendgrouptitle_text=plant,
            legendgroup=plant,
            name="values",
            opacity=0.7,
            marker=dict(color=COLORS[index], size=3),
        )
    )
    peak_valley_figure.add_trace(
        go.Scatter(
            x=df[(df["plant"] == plant) & (df["peak"] == True)]["time"],
            y=df[(df["plant"] == plant) & (df["peak"] == True)]["soil moisture in %"],
            mode="markers",
            legendgroup=plant,
            name="peaks",
            opacity=1,
            marker=dict(color=COLORS[index], size=16, symbol="arrow-bar-down"),
        )
    )
    peak_valley_figure.add_trace(
        go.Scatter(
            x=df[(df["plant"] == plant) & (df["valley"] == True)]["time"],
            y=df[(df["plant"] == plant) & (df["valley"] == True)]["soil moisture in %"],
            mode="markers",
            legendgroup=plant,
            name="valleys",
            opacity=1,
            marker=dict(color=COLORS[index], size=16, symbol="arrow-bar-up"),
        )
    )

peak_valley_figure.update_scenes(
    dict(
        xaxis_title="Time",
        yaxis_title="soil moisture in %",
    )
)

peak_valley_figure.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1,
    xanchor="center",
    x=0.5,
    itemsizing='constant'
))

makedirs("./dist", exist_ok=True)
peak_valley_figure.write_html("./dist/peak_valley.html")

peak_valley_figure.show()

### Pick descends and normalize soil moisture


In [32]:
decending_dfs: list[pd.DataFrame] = []
for plant in PLANTS:
    last_peak = -1
    last_valley = -1
    df_plant = df[df["plant"] == plant].copy()
    df_plant.reset_index(drop=True, inplace=True)
    for row in df_plant.itertuples():
        if row.peak:
            last_peak = row.Index
        if row.valley:
            last_valley = row.Index
            if last_peak > -1 and last_peak < last_valley:
                df_candidate = df_plant.iloc[last_peak:last_valley].copy()
                mininmum_dt = df_candidate["time"].min()
                # offset in days
                df_candidate["offset"] = (
                    (df_candidate["time"] - mininmum_dt).dt.total_seconds() / 3600 / 24
                )
                if (
                    df_candidate.iloc[0]["soil moisture in %"]
                    < df_candidate.iloc[-1]["soil moisture in %"]
                ):
                    continue

                if df_candidate.empty:
                    continue

                # normalize
                df_candidate["soil moisture in %"] = df_candidate[
                    "soil moisture in %"
                ] + (100 - df_candidate["soil moisture in %"].max())

                decending_dfs.append(df_candidate)

    all_decending_dfs = pd.concat(decending_dfs)


## Regression


In [33]:
polyfits: dict = {}
for plant in PLANTS:
    polyfits[plant] = Polynomial.fit(
        all_decending_dfs[all_decending_dfs["plant"] == plant]["offset"],
        all_decending_dfs[all_decending_dfs["plant"] == plant]["soil moisture in %"],
        1,
    )


## Find next watering time


In [34]:
for plant in PLANTS:
    newest_time = df[df["plant"] == plant]["time"].max()
    newest_moisture = df[(df["plant"] == plant) & (df["time"] == newest_time)][
        "soil moisture in %"
    ]
    root_current = (polyfits[plant] - newest_moisture).roots()[0]
    root_minimum = (polyfits[plant] - MINIMUM_MOISTURE[plant]).roots()[0]
    days_after_last_measurement = root_minimum - root_current
    days_after_now = days_after_last_measurement - (datetime.datetime.now() - newest_time).total_seconds() / 3600 / 24
    print(
        f"{plant} will reach {MINIMUM_MOISTURE[plant]} % moisture in {days_after_now} days"
    )

Goldfruchtpalme will reach 10 % moisture in 24.35534842393945 days
Pilea will reach 20 % moisture in 11.200741228447114 days
Drachenbaum will reach 30 % moisture in 50.200811603071145 days


## Plot regression


In [36]:
regression_figure = go.Figure(
    layout=go.Layout(
        title="Moisture Regression",
        width=1280,
        height=720,
        template="plotly_white",
        font=dict(family="Times New Roman", color="#000000"),
    )
)

for index, plant in enumerate(PLANTS):
    regression_figure.add_trace(
        go.Scatter(
            x=all_decending_dfs[all_decending_dfs["plant"] == plant]["offset"],
            y=all_decending_dfs[all_decending_dfs["plant"] == plant][
                "soil moisture in %"
            ],
            mode="markers",
            name=plant,
            opacity=0.5,
            marker=dict(color=COLORS[index], size=3),
        )
    )
    poly_x, poly_y = polyfits[plant].linspace(100)
    regression_figure.add_trace(
        go.Scatter(
            x=poly_x,
            y=poly_y,
            mode="lines",
            name=f"regression {plant}",
            opacity=1,
            line=dict(color=COLORS[index], width=2, dash="longdash"),
        )
    )

regression_figure.update_scenes(
    dict(
        xaxis_title="Time after last watering in days",
        yaxis_title="soil moisture in %",
    )
)

regression_figure.update_layout(
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1,
        xanchor="center",
        x=0.5,
        itemsizing='constant',
    )
)

makedirs("./dist", exist_ok=True)
regression_figure.write_html("./dist/moisture_regression.html")

regression_figure.show()
